# Pre-Defined 

In [1]:
class Binary:
    def __init__(self,value, bit=None):
        if(value[:2]=='0b'):
            value=value[2:]

        if(type(value)!=str):
            raise TypeError(f"TypeError: value \'{value}\' {type(value)} not string")
        for i in value:
            if(i) not in ['1','0']:
                raise ValueError(f"ValueError: value \'{value}\' not in '0' and '1'")        
        self.__value =value
        if(bit is None):
            self.__bit=len(value)
        elif(type(bit) is not int):
            raise ValueError(f"ValueError: value \'{bit}\' not in integer") 
        else:
            self.__bit =bit
            
        Binary.__adder_bit(self)

    def bit(self):
        return self.__bit
    
    def __str__(self):
        return str(self.__value)
    
    def __repr__(self):
        return str(self.__value)
    def __len__(self):
        return self.__bit 
    def circular_shift_left(self,in_place=False):
        lsit_binary=list(self.__value)
        lsit_binary.insert(len(lsit_binary),lsit_binary[0])
        lsit_binary.remove(lsit_binary[0])
        binary=""
        for i in lsit_binary:
            binary=binary+i
        if(in_place):
            self.__value=binary
        return binary
                
          
    def circular_shift_right(self,in_place=False):
        lsit_binary=list(self.__value)
        lsit_binary.insert(0,lsit_binary[len(lsit_binary)-1])
        lsit_binary=lsit_binary[:-1]
        binary=""
        for i in lsit_binary:
            binary=binary+i
        if(in_place):
            self.__value=binary
        return binary

    def invert(self,in_place=False):
        value_invert=self.__value
        value_invert=value_invert.replace('0','#')
        value_invert=value_invert.replace('1','0')
        value_invert=value_invert.replace('#','1')
        if(in_place):
            self.__value=value_invert
        return value_invert
    def update_bit(self,bit):
        self.__bit =bit
        __adder_bit()
        
    def update_value(self,value,bit=None):
        self.__value=value
        if(bit is None):
            self.__bit=len(value)
        else:
            self.__bit =bit
        __adder_bit()

    def to_bin(self):
        return '0b' +self.__value
    
    def to_int(self):
        return int(self.to_bin(),2)
    
    def xor(self,b):
        if(b[:2].lower()=='0b'):
            b=b[2:]
        a=self.__value
        if(len(b)>len(a)):
            for i in range(len(b)-len(a)):
                a='0'+a
        elif(len(a)>len(b)):
            for i in range(len(a)-len(b)):
                b='0'+b
            
        b=b[::-1]
        a=a[::-1]
        
        c=''
        for i,j in zip(a,b):
            if(i=='1' and j=='1' or i=='0' and j=='0'):
                c='0'+c
            else:
                c='1'+c
        return c
    def __adder_bit(self):
        len_value=len(self.__value)
        if(self.bit()>len_value):
            for i in range(self.bit()-len_value):
                self.__value='0'+self.__value
    



In [2]:
def to_binary(text):
    binary_=''
    for i in text:
        binary_=binary_+Binary(bin(ord(i))[2:],8).to_bin()[2:]
    return binary_

In [3]:
def to_text(binary_,char_size=8):
    i=1
    binary_=binary_[::-1]
    block=''
    full_text=''
    block=''
    while(i<len(binary_)):
        block=block+binary_[i-1]
        if(i+1==len(binary_)):
            block=block+binary_[i]
        if(i%(char_size)==0 or i+1==len(binary_)):
            block=block[::-1]
            full_text=chr(int(block,2))+full_text
            block=''
        i=i+1
    return full_text

In [4]:
def byte_to_hex(binary_):
    size=len(binary_)
    hex_=''
    binary_=binary_[::-1]
    oneta=binary_[:4]
    oneta=oneta[::-1]
    hex_=hex_+str(hex(int(oneta,2)))[2:]
    
    if(size>4):
        oneta=binary_[4:]
        oneta=oneta[::-1]
        hex_=str(hex(int(oneta,2)))[2:]+hex_

    return hex_.upper()

# Assignments
Write a code to implement CFB Algorithm taking into account that the size of IV is 64 bit and the number of s bits is 16.



$IV\text{  Initialization vector }$
<br>
$f(x) \text{  can be particular algorithm as AES or DES}$

<br>
$C_{i}=f(E(IV,K))\oplus P_{i}$
<br>
$P_{i}=f(E(IV,K))\oplus C_{i}$
<br>


### Step 1
IV => Encrpypt XOR Key = ENC_IV
### Step 2
First S of ENC_IV XOR First S of Plaintext

<img src="img/de.png" alt="Alternative" />
    

In [5]:
Binary('10',12).xor('10000000')

'000010000010'

In [6]:
import math
def feedback(plaintext,key,IV='',S_bit=16,decrypt_mode=False):
    if(len(key)*8!=64):
        return f'bad input key size {len(key)*8}'
        #note iv can be empty
        #if the size of the key not 64-bit return bad input
    if(len(plaintext)%2 != 0):
        plaintext=plaintext+'x'
        # if the plaintext not even add 'x' to the also end can be ' ' space
    #step 1 IV XOR KEY
    key_binary=to_binary(key)
    IV_binary=to_binary(IV)
    plaintext_binary=to_binary(plaintext)
    
    
    factor =math.ceil(len(plaintext_binary)/S_bit)
    #factor calcalute the number of iteration for the loop
    cipher=''
    # f(x)=>E(iv,key)
    
    
    for i in range(factor):
        shift_register=Binary(key_binary,64).xor(IV_binary)
        # XOR function
        
        XORed=Binary(plaintext_binary[:S_bit],S_bit)
        cipher_subset=XORed.xor(shift_register[:S_bit])
        cipher=cipher+cipher_subset
        
        if(not decrypt_mode):
            shift_register=shift_register[S_bit:]+cipher_subset
            # Shif the first (S_bit) (Removed)
            # add the cipher subset to the end of [IV (XORed) KEY]
        else:
            shift_register=shift_register[S_bit:]+plaintext_binary[:S_bit]
            # in term od decode the plaintext_binary is cipher 
            # added the cipher(plaintext_binary) to recover the plaintext

        plaintext_binary=plaintext_binary[S_bit:]
        # shif the plaintext
    return cipher
    #

# Code

In [7]:
len('cypherpk')*8

64

In [8]:
plaintext='Hello World 2077'
key='cypherpk'
iv='vi' # can be up to 64-bit

## Encode

In [9]:
print(feedback(plaintext,key,iv))

00101011000111000000111100010101000011000101100100110100000101100001000100010101000001110101100101010001010010010101010001001110


In [10]:
ciphertext=to_text(feedback(plaintext,key,iv))
print(ciphertext)

+Y4YQITN


## Decode

In [11]:
print(feedback(ciphertext,key,iv))

01001000011001010110110001101100011011110010000001010111011011110111001001101100011001000010000000110010001100000011011100110111


In [12]:
print(to_text(feedback(ciphertext,key,iv,decrypt_mode=True)))

Hello World 2077


In [13]:
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

In [14]:
KEY=widgets.Text(
    value='feedback',
    placeholder='Key',
    disabled=False
)
IV=widgets.Text(
    value='VI',
    placeholder='Initialization vector ',
    disabled=False
)
P=widgets.Textarea(
    value='',
    placeholder='Plaintext',
    disabled=False
)
C=widgets.Text(
    value='Useless',
    placeholder='Type something',
    disabled=False
)
OUT=widgets.Textarea(
    value='',
    placeholder='Output',
    disabled=False
)
MODE=widgets.RadioButtons(
    options=['Encode','Decode'],
#     value='pineapple',
    description='MODE:',
    disabled=False
)

# Friendly GUI

In [15]:
display(KEY)
display(IV)
display(C)
display(P)
display(MODE)
display(OUT)

button_run = widgets.Button(description="Run")
button_copy = widgets.Button(description="copy")

output = widgets.Output()


display(button_copy)
display(button_run)

def on_button_clicked(b):
    with output:
        if (MODE.value=='Decode'):
            _M=True
        else:
            _M=False
        OUT.value=to_text(feedback(plaintext=P.value,key=KEY.value,IV=IV.value,decrypt_mode=_M))
        # C.value=P.value
        # P.value=OUT.value
        print("Run.")
def on_button_clicked_copy(b):
    with output:
        P.value=OUT.value
        print("copy.")

button_run.on_click(on_button_clicked)
button_copy.on_click(on_button_clicked_copy)


Text(value='feedback', placeholder='Key')

Text(value='VI', placeholder='Initialization vector ')

Text(value='Useless', placeholder='Type something')

Textarea(value='', placeholder='Plaintext')

RadioButtons(description='MODE:', options=('Encode', 'Decode'), value='Encode')

Textarea(value='', placeholder='Output')

Button(description='copy', style=ButtonStyle())

Button(description='Run', style=ButtonStyle())